In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as mlt
import seaborn as sns
mlt.style.use('fivethirtyeight')
from scipy import stats
from scipy.stats import norm,skew
from sklearn.cluster import KMeans

In [6]:
train = pd.read_csv('amex.csv')
train.drop(['Unnamed: 0','mvar16','mvar19','mvar17'],axis=1,inplace=True)
train.head()
test=pd.read_csv('amextest.csv')
test.drop(['Unnamed: 0','mvar16','mvar19','mvar17'],axis=1,inplace=True)
train['mvar47'].loc[train['mvar47'] == 'L'] = 0
train['mvar47'].loc[train['mvar47'] == 'C'] = 1
test['mvar47'].loc[test['mvar47'] == 'L'] = 0
test['mvar47'].loc[test['mvar47'] == 'C'] = 1
kall = train.iloc[:,1:45]
kalltr = kall.dropna()

values = kall.values

from sklearn.preprocessing import Imputer
imputer = Imputer()

transformed_values = imputer.fit_transform(values)

X = pd.DataFrame(transformed_values)
X.columns = kall.columns

for index in X.columns:
    train[index]=X[index]

C:\Users\iramaniparth\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
kall = test.iloc[:,1:45]
kalltr = kall.dropna()

values = kall.values

from sklearn.preprocessing import Imputer
imputer = Imputer()

transformed_values = imputer.fit_transform(values)

X = pd.DataFrame(transformed_values)
X.columns = kall.columns

for index in X.columns:
    test[index]=X[index]

In [8]:
train.head()
train0=np.array(train)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train0[:,1:45], train0[:,45], random_state=0)
print(train0.shape)
train0[:,45]

(80000, 46)


array([0., 1., 1., ..., 1., 0., 1.])

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
clf = LogisticRegression(random_state=0,penalty = 'l1')
clf.fit(train0[:,1:45], train0[:,45])
print("Accuracy on training set: {:.6f}".format(clf.score(X_train, y_train)))
print("Accuracy on test set: {:.6f}".format(clf.score(X_test, y_test)))

Accuracy on training set: 0.800517
Accuracy on test set: 0.804350


In [12]:
ynew = clf.predict_proba(X_test)

In [13]:
ynew

array([[0.84435551, 0.15564449],
       [0.39975296, 0.60024704],
       [0.63053489, 0.36946511],
       ...,
       [0.9630699 , 0.0369301 ],
       [0.89150276, 0.10849724],
       [0.99637674, 0.00362326]])

In [16]:
test0=np.array(test.iloc[:,1:])
ynew = clf.predict_proba(test0)

In [11]:
test=pd.read_csv('amextest.csv')


test['mvar47'].loc[test['mvar47'] == 'L'] = 0
test['mvar47'].loc[test['mvar47'] == 'C'] = 1

C:\Users\iramaniparth\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
d=pd.DataFrame()
d['application_key']=test['application_key']
d['0']=ynew[:,0]
d['1']=ynew[:,1]

In [19]:
y=clf.predict(test0)

In [20]:
d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

In [21]:
sub1 = d[['application_key', 'pred']]

In [22]:
sub1.to_csv('The_miner_league_IITGuwahati_25.csv', header = False, index = False)